In [1]:
import pandas as pd
import numpy as np
df=pd.read_csv("combined.csv")

def convertToFloat(x):
    x=x.strip()
    if x=="-" or x=="NR" or x=='' or x=="." or x=="" or x==' ':
        return 0
    else:
        
        return float(x)
df["rain"]=df["Rain"].apply(convertToFloat)
# import datetime
df.pop("Rain")
aldf=df[df.District=="Alangayam"]
# df["month"]=df["Date"].apply(lambda x:datetime.datetime.strptime(x,"%Y-%m-%d").month)
# df["year"]=df["Date"].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d").year)
# df["day"]=df["Date"].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d").day)

In [2]:
dataset=aldf[["rain"]].values
dataset 

array([[0.],
       [0.],
       [0.],
       ...,
       [0.],
       [0.],
       [0.]])

In [3]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from tensorflow import keras

In [5]:
np.random.seed(7)

dataset=scaler.fit_transform(dataset)

NameError: name 'scaler' is not defined

In [6]:
def CreateDataset(dataset,lookback):
    DataX=[]
    DataY=[]
    for i in range(len(dataset)-lookback-1):
        DataX.append(dataset[i:(lookback+i),0])
        DataY.append(dataset[(lookback+i),0])
    return np.array(DataX),np.array(DataY)

In [7]:
trainX

NameError: name 'trainX' is not defined

In [8]:
trainsize=3287
testsize=365
train, test = dataset[0:trainsize,:], dataset[trainsize:len(dataset),:]

In [9]:
look_back=30
trainX,trainY=CreateDataset(train,look_back)
testX,testY=CreateDataset(test,look_back)
# reshape input to be [samples, time steps, features]
trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

In [218]:
2=@

SyntaxError: invalid syntax (<ipython-input-218-7474edad756f>, line 1)

In [12]:
model = Sequential()

opt = keras.optimizers.Adam()
model.add(keras.layers.Bidirectional(LSTM(4, input_shape=(1, look_back))))
model.add(Dense(1,activation="relu"))
model.compile(loss='mean_absolute_error', optimizer=opt)
model.fit(trainX, trainY, epochs=100, batch_size=1, verbose=2)

Epoch 1/100
3256/3256 - 3s - loss: 2.6821
Epoch 2/100
3256/3256 - 3s - loss: 2.6657
Epoch 3/100
3256/3256 - 3s - loss: 2.6642
Epoch 4/100
3256/3256 - 3s - loss: 2.6630
Epoch 5/100
3256/3256 - 3s - loss: 2.6613
Epoch 6/100
3256/3256 - 3s - loss: 2.6577
Epoch 7/100
3256/3256 - 3s - loss: 2.6511
Epoch 8/100
3256/3256 - 3s - loss: 2.6428
Epoch 9/100
3256/3256 - 3s - loss: 2.6348
Epoch 10/100
3256/3256 - 3s - loss: 2.6239
Epoch 11/100
3256/3256 - 3s - loss: 2.6093
Epoch 12/100
3256/3256 - 3s - loss: 2.5961
Epoch 13/100
3256/3256 - 3s - loss: 2.5860
Epoch 14/100
3256/3256 - 3s - loss: 2.5712
Epoch 15/100
3256/3256 - 3s - loss: 2.5592
Epoch 16/100
3256/3256 - 3s - loss: 2.5365
Epoch 17/100
3256/3256 - 3s - loss: 2.5206
Epoch 18/100
3256/3256 - 3s - loss: 2.5037
Epoch 19/100
3256/3256 - 4s - loss: 2.4835
Epoch 20/100
3256/3256 - 3s - loss: 2.4762
Epoch 21/100
3256/3256 - 3s - loss: 2.4543
Epoch 22/100
3256/3256 - 3s - loss: 2.4464
Epoch 23/100
3256/3256 - 3s - loss: 2.4307
Epoch 24/100
3256/32

In [14]:
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)

from sklearn.metrics import mean_squared_error


ypred=testPredict.reshape(334)
ty=testY.reshape(334)

# rainPredict = scaler.inverse_transform(trainPredict)
# trainY = scaler.inverse_transform([trainY])
# testPredict = scaler.inverse_transform(testPredict)
# testY = scaler.inverse_transform([testY])

NameError: name 'scaler' is not defined

In [16]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(ty,ypred)

2.9558856780657514

In [19]:
ypred[ypred>0].shape

(30,)

In [20]:
ypred.shape

(334,)